In [974]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import numpy as np

import klib as kl


In [975]:
bank_names_mapping = mapping_dict = {
    'abanca': 'abanca',
    'banc sabadell': 'banc sabadell',
    'bancosabadell': 'banc sabadell',
    'bancosantander': 'santander',
    'bancsabadell': 'banc sabadell',
    'sabadell': 'banc sabadell',
    'bank': 'bank',
    'bankinter': 'bankinter',
    'bbva': 'bbva',
    'caixabank': 'caixabank',
    'caixa_bank': 'caixabank',
    'date': 'date',
    'evobanco': 'evobanco',
    'evo_banco': 'evobanco',
    'imagin': 'imagin',
    'ing': 'ing',
    'ing-bank': 'ing',
    'ing_bank': 'ing',
    'kutxabank': 'kutxabank',
    'liberbank': 'liberbank',
    'myinvestor': 'myinvestor',
    'n26': 'n26',
    'n26.com': 'n26',
    None: None,  # handling nan (not a number)
    'openbank': 'openbank',
    'orangebank': 'orangebank',
    'pibank': 'pibank',
    'renaultbank': 'renaultbank',
    'revolut': 'revolut',
    'santander': 'santander',
    'value': 'value'
}

### Web Traffic Cleanup

In [976]:

products_data = pd.read_excel('./data/Account Products.xlsx', sheet_name='web_traffic_accounts')
products_data.Date = pd.to_datetime(products_data.Date)
products_data.set_index('Date', inplace=True)

imputer = KNNImputer(n_neighbors=2)
imputed_data = imputer.fit_transform(products_data)
imputed_data = pd.DataFrame(imputed_data, columns=products_data.columns, index=products_data.index)
products_data = imputed_data.copy()

# get bank name from columns 
products_data.columns = products_data.columns.str.split('.').str[0]

final_cols = products_data.columns

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_data = products_data.select_dtypes(include=numerics)
num_cols_target = numeric_data.columns

# average the n26 and ing columns and combine them
products_data['n26'] = products_data[['n26', 'n26']].mean(axis=1)
products_data['n26'] = products_data[['n26', 'n26']].mean(axis=1)

products_data['ing'] = products_data[['ing', 'ing']].mean(axis=1)
products_data['ing'] = products_data[['ing', 'ing']].mean(axis=1)

# drop the 4th column   and 2nd column 
products_data = products_data.T.drop_duplicates().T

products_data = products_data.rename(columns=bank_names_mapping)

# melt the data so bank names are in a single column
products_data = products_data.reset_index().melt(id_vars='Date', var_name='Bank', value_name='Web Traffic')

products_data = kl.clean_column_names(products_data)

products_data.groupby('bank')['date'].count()

bank
abanca           25
banc sabadell    25
bankinter        25
bbva             25
evobanco         25
ing              25
myinvestor       25
n26              25
openbank         25
revolut          25
santander        25
Name: date, dtype: int64

### Bank Information Cleanup

In [977]:
bank_comps = pd.read_excel('data/Account Products.xlsx', sheet_name='bank_comparison_metrics', header=0)
bank_comps['age'] = 2024 - bank_comps['age of bank']
bank_comps['has_stores'] = bank_comps['number of branches'] > 0
bank_comps['has_stores'] = bank_comps['has_stores'].astype(int)
bank_comps['founded_before_2000'] = bank_comps['age of bank'] < 2000
bank_comps['founded_before_2000'] = bank_comps['founded_before_2000'].astype(int)
bank_comps['bank'] = bank_comps.website.str.split('.').apply(lambda x: x[0])

bank_comps = bank_comps[['bank', 'number of employee', 'assets under management', 'number of branches', 'age', 'has_stores', 'founded_before_2000']]

bank_comps = kl.clean_column_names(bank_comps)

bank_comps.head()

,bank,number_of_employee,assets_under_management,number_of_branches,age,has_stores,founded_before_2000
0,abanca,5946,72148000000,690,13,1,0
1,bancosantander,212764,1117000000000,8518,167,1,1
2,bancsabadell,19316,253000000000,1594,143,1,1
3,bankinter,6138,83300000000,523,59,1,1
4,bbva,121486,775000000000,1800,167,1,1


In [998]:
# add a column for the sum of web traffic for each bank
web_traffic = products_data.groupby('bank')['web_traffic'].sum()
bank_comps['web_traffic'] = bank_comps.bank.map(dict(zip(web_traffic.index, web_traffic.values)))
bank_comps.web_traffic.unique()

array([1357941.25029154,  976275.86881283,  734644.19632741,
        275543.46320441,  992082.84725461,              nan,
        169628.42402407,  701693.01919068,  229436.06865739,
        203100.78870238,  424391.86973633,  146959.2854638 ])

In [1002]:
bank_comps.to_csv('data/bank_comps.csv', index=False)

### Cross Visitation Classification

In [978]:
cross_visitation_data = pd.read_excel('data/Account Products.xlsx', sheet_name='cross_visitation', header=1)

cross_visitation_data = kl.data_cleaning(cross_visitation_data)

# get numerica columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_data = cross_visitation_data.select_dtypes(include=numerics)
num_cols = numeric_data.columns

# impute missing values
imputer = KNNImputer(n_neighbors=2)
imputed_data = imputer.fit_transform(cross_visitation_data[num_cols])
imputed_data = pd.DataFrame(imputed_data, columns=num_cols, index=cross_visitation_data.index)
cross_visitation_data[num_cols] = imputed_data.copy()
cross_visitation_data.head()

# reformat column names
all_cols = cross_visitation_data.columns
cross_visitation_data.columns = cross_visitation_data.columns.str.split('_').str[0]

# melt the data so bank names are in a single column
cross_visitation_data = cross_visitation_data.reset_index().melt(id_vars='date', var_name='Bank', value_name='Cross Visitation')

cross_visitation_data = kl.clean_column_names(cross_visitation_data)

# drop where Bank is == inde
cross_visitation_data = cross_visitation_data.loc[~cross_visitation_data.bank.isin(['index'])]

cross_visitation_data.groupby('bank')['date'].count()

Shape of cleaned data: (25, 16) - Remaining NAs: 25


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.0 MB (-nan%)



/Users/andrewbennett/Library/Python/3.9/lib/python/site-packages/klib/utils.py:142: RuntimeWarning: invalid value encountered in scalar divide
  mem_perc = round(100 * mem_change / data_mem, 2)


bank
abanca            25
bancosantander    25
bancsabadell      25
bankinter         25
bbva              25
caixabank         25
evobanco          25
imagin            25
ing               25
kutxabank         25
myinvestor        25
n26               25
openbank          25
pibank            25
revolut           25
Name: date, dtype: int64

### Account Ranking Preprocessing

In [979]:

file_path = './data/Account Products.xlsx'
df = pd.read_excel(file_path, sheet_name='ranking_position', skiprows=7)


df['Unnamed: 0'] = df['Unnamed: 0'].fillna(method='ffill')

# Rename the columns for clarity:
df.columns = ['Category', 'Bank Name'] + list(df.columns[2:])

transformed_df = pd.melt(df, id_vars=['Category', 'Bank Name'], var_name='Date', value_name='Rank')
transformed_df['Date'] = pd.to_datetime(transformed_df['Date'])

# drop category column
transformed_df = transformed_df.drop(columns='Category')

# group by bank name and date and average the rank
transformed_df = transformed_df.groupby(['Bank Name', 'Date']).mean().reset_index()


transformed_df = kl.clean_column_names(transformed_df)

# rename bank_name to bank
transformed_df = transformed_df.rename(columns={'bank_name': 'bank'})

rank_df = transformed_df[['bank', 'date', 'rank']]

# lowercase the bank names
rank_df['bank'] = rank_df['bank'].str.lower()

# map the bank names
rank_df['bank'] = rank_df['bank'].map(bank_names_mapping)

rank_df.groupby('bank')['date'].count()

/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_46912/502252227.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Unnamed: 0'] = df['Unnamed: 0'].fillna(method='ffill')


bank
abanca           26
banc sabadell    26
bankinter        26
bbva             26
caixabank        26
evobanco         26
imagin           26
ing              26
liberbank        26
myinvestor       26
n26              26
openbank         26
orangebank       26
pibank           26
renaultbank      26
revolut          26
santander        26
Name: date, dtype: int64

### Bank Incentives Clean Up

In [980]:
incentives_data = pd.read_csv('data/cleaned_incentives_data.csv')

In [981]:
# ffill then bfill the missing  values
incentives_data = incentives_data.ffill().bfill()

#  cast the date column to datetime
incentives_data.date = pd.to_datetime(incentives_data.date)

incentives_data.groupby('bank')['date'].count()

/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_46912/2608812180.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  incentives_data.date = pd.to_datetime(incentives_data.date)


bank
abanca           36
banc sabadell    36
bankinter        36
bbva             36
revolut          36
santander        36
Name: date, dtype: int64

### News Release Clean Up

In [982]:
news_data = pd.read_csv('text_files/tucapital-news_with_text-translated.csv')
news_data

,Unnamed: 0,title,url,date,text
0,0,"Tomorrow, last day of the account at 2.89% APR...",https://www.tucapital.es/depositos/manana-ulti...,23/04/2024,"Tomorrow, April 24, 2024, the marketing period..."
1,1,ING gives you up to €120 if you direct your pa...,https://www.tucapital.es/cuentas/ing-te-da-has...,22/04/2024,"ING now asks, in addition to direct debiting a..."
2,2,Banco BIG deposits: slight increase in normal ...,https://www.tucapital.es/depositos/depositos-d...,22/04/2024,Banco BIG remodels its deposit offer with slig...
3,3,The future of Bitcoin: predictions for the nex...,https://www.tucapital.es/blogs/guias/el-futuro...,22/04/2024,"Since its launch, Bitcoin has seen positive an..."
4,4,Eco: Cetelem only allows maximum daily transfe...,https://www.tucapital.es/blogs/eco/eco-cetelem...,19/04/2024,"Due to the disappearance of Orange Bank, and t..."
...,...,...,...,...,...
1020,1020,Pibank and Pichincha maintain your deposit at ...,https://www.tucapital.es/cuentas/pibank-y-pich...,04/01/2021,"Pibank, the online office of Banco Pichincha, ..."
1021,1021,"New year, new taxes. This is how the Treasury ...",https://www.tucapital.es/blogs/noticias/ano-nu...,04/01/2021,I wish that with this first post of the year w...
1022,1022,"Big Bank, RenaultBank...and OrangeBank?, new a...",https://www.tucapital.es/cuentas/banco-big-ren...,31/12/2020,"As we told you yesterday (see here), 2020 has ..."
1023,1023,"Goodbye 2020, black year of banking fees. News...",https://www.tucapital.es/blogs/noticias/adios-...,30/12/2020,2020 has been a black year. It has been the ye...


In [983]:
bank_names = list(products_data.bank.unique())
bank_names.remove('ing')
bank_names.append('ing_bank')
bank_names

['abanca',
 'n26',
 'banc sabadell',
 'bbva',
 'revolut',
 'openbank',
 'myinvestor',
 'bankinter',
 'evobanco',
 'santander',
 'ing_bank']

In [984]:
# search for each of the bank names in the news data

news_data['text_nospace'] = news_data['text'].str.replace('ING', 'ing_bank')
news_data['text_nospace'] = news_data['text_nospace'].str.replace(' ', '').str.lower()


for bank in bank_names:
    news_data[bank] = 0

for index, row in news_data.iterrows():
    for bank in bank_names:
        if bank in row['text_nospace']:
            news_data.at[index, bank] = news_data.at[index, bank] + 1

news_data.head()


,Unnamed: 0,title,url,date,text,text_nospace,abanca,n26,banc sabadell,bbva,revolut,openbank,myinvestor,bankinter,evobanco,santander,ing_bank
0,0,"Tomorrow, last day of the account at 2.89% APR...",https://www.tucapital.es/depositos/manana-ulti...,23/04/2024,"Tomorrow, April 24, 2024, the marketing period...","tomorrow,april24,2024,themarketingperiodforthe...",0,0,0,0,0,0,0,0,0,0,0
1,1,ING gives you up to €120 if you direct your pa...,https://www.tucapital.es/cuentas/ing-te-da-has...,22/04/2024,"ING now asks, in addition to direct debiting a...","ing_banknowasks,inadditiontodirectdebitingapay...",0,0,0,0,0,0,0,0,0,0,1
2,2,Banco BIG deposits: slight increase in normal ...,https://www.tucapital.es/depositos/depositos-d...,22/04/2024,Banco BIG remodels its deposit offer with slig...,bancobigremodelsitsdepositofferwithslightincre...,0,0,0,0,0,0,1,0,0,0,1
3,3,The future of Bitcoin: predictions for the nex...,https://www.tucapital.es/blogs/guias/el-futuro...,22/04/2024,"Since its launch, Bitcoin has seen positive an...","sinceitslaunch,bitcoinhasseenpositiveandnegati...",0,0,0,0,1,0,0,0,0,0,0
4,4,Eco: Cetelem only allows maximum daily transfe...,https://www.tucapital.es/blogs/eco/eco-cetelem...,19/04/2024,"Due to the disappearance of Orange Bank, and t...","duetothedisappearanceoforangebank,andthetransf...",0,1,0,0,0,0,0,0,0,0,0


In [985]:
news_data['month_date'] = pd.to_datetime(news_data['date'])
# set month date to the first day of the month
news_data['month_date'] = news_data['month_date'].dt.to_period('M').dt.to_timestamp()
news_data = news_data.groupby('month_date')[bank_names].sum()

# melt the df to have the bank names as a column
news_data = news_data.reset_index().melt(id_vars='month_date', value_vars=bank_names, var_name='bank', value_name='mentions')

# replace the bank names with the correct names
news_data.bank = news_data.bank.map(bank_names_mapping)


news_data.columns = ['date', 'bank', 'mentions']

news_data.groupby('bank')['date'].count()



/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_46912/2433273592.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  news_data['month_date'] = pd.to_datetime(news_data['date'])


bank
abanca           41
banc sabadell    41
bankinter        41
bbva             41
evobanco         41
ing              41
myinvestor       41
n26              41
openbank         41
revolut          41
santander        41
Name: date, dtype: int64

### Google Trends Data

In [986]:
data = pd.read_csv('data/google_trends.csv')
data = kl.clean_column_names(data)

data.head()

,unnamed_0,abanca,santander,sabadell,bbva,bankinter,caixa_bank,evo_banco,n26,openbank
0,12/5/2021,6,58,28,41,11,7,2,1,4
1,12/12/2021,8,71,38,49,13,7,3,1,4
2,12/19/2021,8,65,33,47,12,6,3,1,4
3,12/26/2021,9,68,36,47,12,6,3,1,5
4,1/2/2022,8,61,31,46,11,6,3,1,5


In [987]:
# map column names to the correct bank names

data.columns = data.columns.map(bank_names_mapping)

data.columns = [            'date',        'abanca',     'santander', 'banc sabadell',
                'bbva',     'bankinter',     'caixabank',      'evobanco',
                 'n26',      'openbank']
# cast the date column to datetime
data.date = pd.to_datetime(data.date)

# melt the data so bank names are in a single column
search_interest = data.melt(id_vars='date', var_name='bank', value_name='search_interest')

# create a month_date column
search_interest['date'] = search_interest['date'].dt.to_period('M').dt.to_timestamp()

search_interest = search_interest.groupby(['date', 'bank'])['search_interest'].mean().reset_index()

search_interest.groupby('bank')['date'].count()


bank
abanca           25
banc sabadell    25
bankinter        25
bbva             25
caixabank        25
evobanco         25
n26              25
openbank         25
santander        25
Name: date, dtype: int64

### Data Consolidation by Bank

In [988]:
dfs = [products_data, bank_comps, cross_visitation_data, rank_df, incentive_increases, apr_increases, news_data]

# map the bank names to a single name

for df in dfs:
    print(df.columns)
    df['bank'] = df['bank'].map(bank_names_mapping)

Index(['date', 'bank', 'web_traffic'], dtype='object')
Index(['bank', 'number_of_employee', 'assets_under_management',
       'number_of_branches', 'age', 'has_stores', 'founded_before_2000'],
      dtype='object')
Index(['date', 'bank', 'cross_visitation'], dtype='object')
Index(['bank', 'date', 'rank'], dtype='object')
Index(['date', 'bank', 'incentive'], dtype='object')
Index(['date', 'bank', 'apr'], dtype='object')
Index(['date', 'bank', 'mentions'], dtype='object')


In [989]:
# list all dfs
bank_names = products_data.bank.unique()
dates = pd.date_range(start='2021-12-01', end='2023-12-01', freq='MS')

banks_dates = pd.DataFrame([(bank, date) for bank in bank_names for date in dates], columns=['bank', 'date'])

all_data = pd.merge(banks_dates, products_data, on=['bank', 'date'], how='left')
print('here')
#all_data = pd.merge(all_data, bank_comps, on=['bank'], how='left')
all_data = pd.merge(all_data, cross_visitation_data, on=['bank', 'date'], how='left')
print('here')
all_data = pd.merge(all_data, search_interest, on=['bank', 'date'], how='left')

all_data = pd.merge(all_data, rank_df[['rank','bank', 'date']], on=['bank', 'date'], how='left')

all_data = pd.merge(all_data, incentives_data, on=['bank', 'date'], how='left')

all_data = pd.merge(all_data, news_data, on=['bank', 'date'], how='left')

all_data.head()


here
here


,bank,date,web_traffic,cross_visitation,search_interest,rank,incentive,apr,mentions
0,abanca,2021-12-01,15202.983835,0.077994,7.75,4.0,150.0,0.0,1
1,abanca,2022-01-01,42183.913207,0.062500,7.80,4.0,150.0,0.0,1
2,abanca,2022-02-01,34498.153115,0.052288,7.25,4.0,150.0,0.0,2
3,abanca,2022-03-01,34546.319021,0.061503,6.75,NaN,150.0,0.0,2
4,abanca,2022-04-01,40508.288420,0.050222,6.75,NaN,300.0,0.0,2


In [990]:
# ffill and bfill these columns 'category', 'rank'
all_data['rank'] = all_data['rank'].ffill().bfill()
all_data['apr'] = all_data['apr'].ffill().bfill()
all_data['incentive'] = all_data['incentive'].ffill().bfill()

#fill category with mode
# all_data['category_best_salary_accounts_helmycash'] = all_data['category_best_salary_accounts_helmycash'].fillna(all_data['category_best_salary_accounts_helmycash'].mode()[0])
# all_data['category_best_savings_accounts_helpmycash'] = all_data['category_best_savings_accounts_helpmycash'].fillna(all_data['category_best_savings_accounts_helpmycash'].mode()[0])

# all_data['category_best_salary_accounts_helmycash'] = all_data['category_best_salary_accounts_helmycash'].astype(int)
# all_data['category_best_savings_accounts_helpmycash'] = all_data['category_best_savings_accounts_helpmycash'].astype(int)

all_data.head()

,bank,date,web_traffic,cross_visitation,search_interest,rank,incentive,apr,mentions
0,abanca,2021-12-01,15202.983835,0.077994,7.75,4.0,150.0,0.0,1
1,abanca,2022-01-01,42183.913207,0.062500,7.80,4.0,150.0,0.0,1
2,abanca,2022-02-01,34498.153115,0.052288,7.25,4.0,150.0,0.0,2
3,abanca,2022-03-01,34546.319021,0.061503,6.75,4.0,150.0,0.0,2
4,abanca,2022-04-01,40508.288420,0.050222,6.75,4.0,300.0,0.0,2


In [991]:
# fill the missing search interest values with the iterative imputer
imputer = KNNImputer(n_neighbors=2)
imputed_data = imputer.fit_transform(all_data[['search_interest', 'mentions']])
imputed_data = pd.DataFrame(imputed_data, columns=['search_interest', 'mentions'], index=all_data.index)
all_data[['search_interest', 'mentions']] = imputed_data.copy()

In [992]:
all_data.drop_duplicates().groupby('bank')['date'].count()

bank
abanca           25
banc sabadell    25
bankinter        25
bbva             25
evobanco         25
ing              25
myinvestor       25
n26              25
openbank         25
revolut          25
santander        25
Name: date, dtype: int64

In [993]:
# save the data to a csv file

all_data.to_csv('data/regression_clean_data.csv', index=False)

In [644]:
all_data.isna().sum()

bank                                          0
date                                          0
web_traffic                                   0
number_of_employee                            0
assets_under_management                       0
number_of_branches                            0
age                                           0
has_stores                                    0
founded_before_2000                           0
cross_visitation                              0
search_interest                               0
rank                                          0
category_best_salary_accounts_helmycash       0
category_best_savings_accounts_helpmycash     0
incentive                                     0
apr                                           0
mentions                                     75
dtype: int64